In [ ]:
import sqlite3
import random
import itertools
import subprocess
import os
import shlex
import time
import pickle
import joblib

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from matplotlib.colors import Normalize

from PIL import Image, ImageDraw, ImageFont

In [ ]:
dataset_size = 120000
batch_size = 1000
num_images = dataset_size

In [ ]:
filepaths = []

with open("/home/rte/data/paths/all_converted_jpg_images_shuf.txt", "r") as f:
    lines = f.readlines()
    print("length:",len(lines))
    print(lines[0])
for l in lines:
    # substrings = l.rsplit(",", 1)
    filepaths.append(l.strip())
    # image_ids.append(substrings[1].strip())
    
# image_folder = "/mnt/hd2/images/all/"
image_folder = "/home/rte/data/images/web/120k/"

# get the image paths for all images that we have features for
images = []

for i, f in enumerate(filepaths[:]):
    images.append(image_folder + f)
print(len(images))
print(images[:10])

# resize to make sure we don't try and read more images than we have
filepaths = filepaths[:dataset_size]
print(len(filepaths))

In [ ]:
x = [] # images as arrays
dim = 256 # size of one dimension
f = dim * dim # size of vector

In [ ]:
def load_img(path, dim):
    im = Image.open(path).convert('RGB')
    img_x, img_y = im.size

    # im.show()

    if False:
        x_bigger = True if img_x > img_y else False
        ratio = img_x / img_y

        if x_bigger:
            factor = float(dim) / img_y 
        else:
            factor = float(dim) / img_x
        new_x = int(img_x * factor)
        new_y = int(img_y * factor)

        resized = im.resize((new_x, new_y), Image.ANTIALIAS)
        # resized.show()

        left = (new_x - dim)/2
        top = (new_y - dim)/2
        right = (new_x + dim)/2
        bottom = (new_y + dim)/2

        cropped = resized.crop((left, top, right, bottom))
        cropped.show()

#     print("factor", factor)
#     print("ratio", ratio)
#     print(x_bigger)
#     print("img_x", img_x)
#     print("img_y", img_y)
#     print(new_x, new_y)
#     print("cropped", cropped.size)

    # just resize to fill the box
    just_resize = im.resize((256,256), Image.ANTIALIAS)
#     just_resize.show()

    if False:
        if x_bigger:
            factor = float(dim) / img_x
        else:
            factor = float(dim) / img_y
        new_x = int(img_x * factor)
        new_y = int(img_y * factor)
    #     print("factor", factor)
    #     print("img_x", img_x)
    #     print("img_y", img_y)
        crop_resized = im.resize((new_x, new_y))

        # pad
        padded = Image.new("RGB", (dim, dim))
        padded.paste(crop_resized, ((dim - new_x)//2, (dim - new_y)//2))
        padded.show()
    
    return just_resize

In [ ]:
mean = np.array([224.03429894, 223.6761458, 223.62304944])

In [ ]:
# multiprocessing read images
from multiprocessing import Pool

def image_worker(filename):
    img_data = np.array(load_img(filename, dim)) - mean
    return img_data

In [ ]:
num_workers = 8
image_batch = images[:5000]
start = time.time()
with Pool(num_workers) as p:
    q = p.map(image_worker, image_batch)
print(f"time taken: {time.time() - start}")

In [ ]:
img = Image.fromarray(q[0], 'RGB')
img.show()